# Charting the different "Alpha" indices

The NSE has quite a few "strategy" [indices](https://www.nseindia.com/products/content/equities/indices/strategic_indices.htm) floating about. Here is how you explore then on pluto.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()

In [ ]:
#what are the start and end-dates for which total-return (TR) indices are available for 'alpha' incies?

#documentation: https://plutopy.readthedocs.io/en/latest/Indices.html

alphaIndices <- indices$NseTimeSeries() %>%
    filter(NAME %like% '%alpha% TR') %>%
    group_by(NAME) %>%
    summarize(ST = min(TIME_STAMP, na.rm=T), ET = max(TIME_STAMP, na.rm=T)) %>%
    arrange(desc(ST)) %>%
    collect()

alphaIndices %>% print()

In [ ]:
# it looks like most of them are covered through 2005-04-01

startDate <- as.Date("2005-04-01")

dailies <- NULL
for(index in alphaIndices$NAME){
    pxDt <- indices$NseTimeSeries() %>%
        filter(NAME == index & TIME_STAMP >= startDate) %>%
        select(TIME_STAMP, CLOSE) %>%
        collect()
    
    dailies <- merge.xts(dailies, dailyReturn(xts(pxDt$CLOSE, pxDt$TIME_STAMP)))
}

names(dailies) <- alphaIndices$NAME

In [ ]:
Common.PlotCumReturns(dailies, "Alpha Indices", NA)

### tl;dr

1. If you want a quick recovery from panic drawdowns, go with the one with "quality" attached. 
1. The "best" in terms of performance is the NIFTY ALPHA LOW-VOLATILITY 30 TR [index](https://www.niftyindices.com/indices/equity/strategy-indices/nifty-alpha-low-volatility-30). 
1. Caveat: it also includes stocks from the MIDCAP-50 index.

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)